https://adventofcode.com/2022/day/21

In [1]:
import re
from operator import add, sub, mul, floordiv, truediv, eq
from dataclasses import dataclass, field
from typing import Optional, Self, Callable

operators = {'-': sub,
             '*': mul,
             '+': add,
             '/': truediv}

inverse_operators = {'-': '+',
                     '*': '/',
                     '+': '-',
                     '/': '*'}

@dataclass
class Monkey():
    name: str
    dependencies: Optional[list[str]]=None
    operator: Optional[str]=None
    number: Optional[int]=None
    op: Optional[Callable]=field(init=False)
        
    def __post_init__(self):
        if self.operator:
            self.op = operators[self.operator]
        
    def get_value(self,
                  monkeys: dict[Self]) -> int:
        if self.number is None:
            self.number = self.op(*[monkeys[d].get_value(monkeys)
                                    for d in self.dependencies])
        return self.number
    
    def all_dependencies(self,
                         monkeys: dict[Self]) -> list[str]:
        dependencies = []
        queue = [self]
        while queue:
            m = queue.pop()
            d = m.dependencies
            if d:
                for d_ in d:
                    dependencies.append(d_)
                    queue.append(monkeys[d_])
        return dependencies

op_func_re = re.compile('([a-z]{4}):\s([a-z]{4})\s([+\-\/*])\s([a-z]{4})')
int_func_re = re.compile('([a-z]{4}):\s(\d+)')

def read_data(filename:str):
    with open(filename, 'r') as f:
        data = [line.strip() for line in f.readlines()]
    return data

def solve(c: int,
          op: str,
          a: Optional[int]=None,
          b: Optional[int]=None) -> int:
    """solve equation of the form operation(a, b) = c
       where either a or b might be unknown"""
    if a is None: # solve for a
        if op == '/':
            func = lambda x: x - (b*c)
        else:
            operator = operators[op]
            func = lambda x: operator(x, b) - c
    else: # solve for b
        if op == '/':
            func = lambda x: a - (x*c)
        else:
            operator = operators[op]
            func = lambda x: operator(a, x) - c
    x = 0
    y = func(x)
    y_prime = (func(x-1) - func(x+1))/2
    return x + y / y_prime # Newton's method

def solve_monkeys(filename: str) -> list[Monkey]:
    data = read_data(filename)
    monkeys = {}
    for line in data:
        if c := re.findall(op_func_re, line):
            a, d1, op, d2 = c[0]
            monkeys[a] = Monkey(name=a, dependencies=[d1, d2], operator=op)
        elif c := re.findall(int_func_re, line):
            a, n = c[0]
            monkeys[a] = Monkey(name=a, number=int(n))
    return monkeys

def solve_part_1(filename: str) -> int:
    monkeys = solve_monkeys(filename)
    return int(monkeys['root'].get_value(monkeys=monkeys))

def solve_part_2(filename: str) -> int:
    monkeys = solve_monkeys(filename)
    monkeys['root'].operator = '-'
    monkeys['root'].op = sub
    target = 0
    m = monkeys['root']
    while m.dependencies:
        d1, d2 = [monkeys[d] for d in m.dependencies]
        if 'humn' not in d1.all_dependencies(monkeys) and d1.name != 'humn':
            target = solve(target, op=m.operator, a=d1.get_value(monkeys))
            m = d2
        else:
            target = solve(target, op=m.operator, b=d2.get_value(monkeys))
            m = d1
    return int(target)

In [2]:
filename = "../example_data/day21_example_data.txt"
solve_part_1(filename)

152

In [3]:
filename = "../data/day21_data.txt"
solve_part_1(filename)

145167969204648

In [4]:
filename = "../example_data/day21_example_data.txt"
solve_part_2(filename)

301

In [5]:
filename = "../data/day21_data.txt"
solve_part_2(filename)

3330805295850